In [123]:
import torch
import time

from torchrl.envs import GymEnv, StepCounter, TransformedEnv
from tensordict.nn import TensorDictModule as Mod, TensorDictSequential as Seq
from torchrl.modules import EGreedyModule, MLP, QValueModule
from torchrl.collectors import SyncDataCollector
from torchrl.data import LazyTensorStorage, ReplayBuffer
from torch.optim import Adam
from torchrl.objectives import DQNLoss, SoftUpdate
from torchrl._utils import logger as torchrl_logger
from torchrl.record import CSVLogger, VideoRecorder
from torchrl.modules import QValueActor
from torchrl.data import CompositeSpec

torch.manual_seed(0)


In [124]:
# Define the environment
env = TransformedEnv(GymEnv("CartPole-v1"), StepCounter())
env.set_seed(0)

/home/zosov/anaconda3/envs/final-project/lib/python3.11/site-packages/torchrl/envs/common.py:2989: DeprecationWarning: Your wrapper was not given a device. Currently, this value will default to 'cpu'. From v0.5 it will default to `None`. With a device of None, no device casting is performed and the resulting tensordicts are deviceless. Please set your device accordingly.
  warnings.warn(


795726461

In [125]:
class MICOMLPNetwork(torch.nn.Module):
    def __init__(self,
                 in_features,
                 activation_class, 
                 encoder_out_features,
                 mlp_out_features,
                 encoder_num_cells = None,
                 mlp_num_cells = None):
        super(MICOMLPNetwork, self).__init__()

        self.activation = activation_class()

        if encoder_num_cells is None:
            encoder_num_cells = []
        layers_sizes = [in_features] + encoder_num_cells + [encoder_out_features]

        self.encoder = torch.nn.ModuleList()
        for i in range(len(layers_sizes) - 1):
            self.encoder.append(torch.nn.Linear(layers_sizes[i], layers_sizes[i+1]))

        if mlp_num_cells is None:
            mlp_num_cells = []

        layers_sizes = [encoder_out_features] + mlp_num_cells + [mlp_out_features.item()]

        self.q_net = torch.nn.ModuleList()
        for i in range(len(layers_sizes) - 1):
            self.q_net.append(torch.nn.Linear(layers_sizes[i], layers_sizes[i+1]))
        
    
    def forward(self, x):
        for i in range(len(self.encoder)):
            x = self.activation(self.encoder[i](x))

        representation = x

        for i in range(len(self.q_net)-1):
            x = self.activation(self.q_net[i](x))

        return self.q_net[-1](x), representation

In [126]:
value_mlp = MICOMLPNetwork(
    in_features=4,
    activation_class=torch.nn.ReLU,
    encoder_out_features=3,
    mlp_out_features=env.action_spec.shape[-1],
    encoder_num_cells=[64],
    mlp_num_cells=[64]
)

value_mlp

MICOMLPNetwork(
  (activation): ReLU()
  (encoder): ModuleList(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=3, bias=True)
  )
  (q_net): ModuleList(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [127]:
q_values, representation = value_mlp(torch.randn(1, 4))
print(q_values)
print(representation)

tensor([[-0.0332,  0.2120]], grad_fn=<AddmmBackward0>)
tensor([[0.4680, 0.0689, 0.4621]], grad_fn=<ReluBackward0>)


In [128]:
value_net = Mod(value_mlp, 
                in_keys=["observation"], 
                out_keys=["action_value", "representation"])
value_net

TensorDictModule(
    module=MICOMLPNetwork(
      (activation): ReLU()
      (encoder): ModuleList(
        (0): Linear(in_features=4, out_features=64, bias=True)
        (1): Linear(in_features=64, out_features=3, bias=True)
      )
      (q_net): ModuleList(
        (0): Linear(in_features=3, out_features=64, bias=True)
        (1): Linear(in_features=64, out_features=2, bias=True)
      )
    ),
    device=cpu,
    in_keys=['observation'],
    out_keys=['action_value', 'representation'])

In [129]:
# policy = Seq(value_net, 
#              QValueModule(spec=env.action_spec))
# policy

policy = QValueActor(
    module=value_net,
    spec=CompositeSpec(action= env.specs["input_spec", "full_action_spec", "action"]),
    in_keys=["observation"],
)
policy

QValueActor(
    module=ModuleList(
      (0): TensorDictModule(
          module=MICOMLPNetwork(
            (activation): ReLU()
            (encoder): ModuleList(
              (0): Linear(in_features=4, out_features=64, bias=True)
              (1): Linear(in_features=64, out_features=3, bias=True)
            )
            (q_net): ModuleList(
              (0): Linear(in_features=3, out_features=64, bias=True)
              (1): Linear(in_features=64, out_features=2, bias=True)
            )
          ),
          device=cpu,
          in_keys=['observation'],
          out_keys=['action_value', 'representation'])
      (1): QValueModule()
    ),
    device=cpu,
    in_keys=['observation'],
    out_keys=['representation', 'action', 'action_value', 'chosen_action_value'])

In [130]:
# Define the exploration step (e-greedy policy)
exploration_module = EGreedyModule(
    env.action_spec, 
    annealing_num_steps=100_000, 
    eps_init=0.1,
)
policy_explore = Seq(policy, 
                     exploration_module)
policy_explore

TensorDictSequential(
    module=ModuleList(
      (0): QValueActor(
          module=ModuleList(
            (0): TensorDictModule(
                module=MICOMLPNetwork(
                  (activation): ReLU()
                  (encoder): ModuleList(
                    (0): Linear(in_features=4, out_features=64, bias=True)
                    (1): Linear(in_features=64, out_features=3, bias=True)
                  )
                  (q_net): ModuleList(
                    (0): Linear(in_features=3, out_features=64, bias=True)
                    (1): Linear(in_features=64, out_features=2, bias=True)
                  )
                ),
                device=cpu,
                in_keys=['observation'],
                out_keys=['action_value', 'representation'])
            (1): QValueModule()
          ),
          device=cpu,
          in_keys=['observation'],
          out_keys=['representation', 'action', 'action_value', 'chosen_action_value'])
      (1): EGreedyModule()
   

In [131]:
# Define how to collect the data (experiences)
init_rand_steps = 5000 # warm-up steps
frames_per_batch = 100
optim_steps = 10
replay_capacity = 100_000

# NOTE: collector will gather rollouts continously
# If the current trajectory ends, it will start a new one
# NOTE: the rollout gotten from the collector is a dictionary
# that defines the sate and next state as a tensor with a batch dimension in the begining
# for example a rollout of 10 steps will have a tensor of observation of 10 in the batch dimension
# and the next will also have 10 which are all the tensors of the next state
# Practically, next is as you will shift the tensor of observation by one step
# collector = SyncDataCollector(
#     env,
#     policy_explore,
#     frames_per_batch=frames_per_batch,
#     total_frames=500_100,
#     init_random_frames=init_rand_steps,
# )
# rb = ReplayBuffer(storage=LazyTensorStorage(replay_capacity))

In [132]:
# Define the recording and logging
path = "./training_loop"
logger = CSVLogger(exp_name="dqn", log_dir=path, video_format="mp4")
video_recorder = VideoRecorder(logger, tag="video")
record_env = TransformedEnv(
    GymEnv("CartPole-v1", from_pixels=True, pixels_only=False), video_recorder
)

/home/zosov/anaconda3/envs/final-project/lib/python3.11/site-packages/torchrl/envs/common.py:2989: DeprecationWarning: Your wrapper was not given a device. Currently, this value will default to 'cpu'. From v0.5 it will default to `None`. With a device of None, no device casting is performed and the resulting tensordicts are deviceless. Please set your device accordingly.
  warnings.warn(


In [133]:
# collector = SyncDataCollector(
#     env,
#     policy_explore,
#     frames_per_batch=10,
#     total_frames=500_100,
#     init_random_frames=10000,
# )

collector = SyncDataCollector(
    create_env_fn=env,
    policy=policy_explore,
    frames_per_batch=10,
    total_frames=100,
    device="cpu",
    storing_device="cpu",
    max_frames_per_traj=-1
)
# NOTE: IMPORTANTISIMO en las primeras iteraciones no se usa la policy, entonces representation se configura
# a zero, por lo que el primer batch de datos no tiene representation
# Tengo que hacer el warm-up de otra manera (ojo con esto)

for data in collector:
    print(data['representation'])
    break

tensor([[0.1717, 0.1063, 0.0000],
        [0.1425, 0.0952, 0.0061],
        [0.0912, 0.0871, 0.0460],
        [0.0490, 0.0752, 0.0896],
        [0.0580, 0.0906, 0.1324],
        [0.0857, 0.0904, 0.1998],
        [0.1152, 0.0904, 0.2713],
        [0.1495, 0.0832, 0.3370],
        [0.1411, 0.0853, 0.3108],
        [0.1740, 0.0766, 0.3788]])


In [134]:
data['representation']

tensor([[0.1717, 0.1063, 0.0000],
        [0.1425, 0.0952, 0.0061],
        [0.0912, 0.0871, 0.0460],
        [0.0490, 0.0752, 0.0896],
        [0.0580, 0.0906, 0.1324],
        [0.0857, 0.0904, 0.1998],
        [0.1152, 0.0904, 0.2713],
        [0.1495, 0.0832, 0.3370],
        [0.1411, 0.0853, 0.3108],
        [0.1740, 0.0766, 0.3788]])

In [135]:
from torchrl.data import SliceSampler
from torchrl.data import TensorDictReplayBuffer

size = 100
rb = TensorDictReplayBuffer(
    storage=LazyTensorStorage(size),
    sampler=SliceSampler(traj_key=("collector","traj_ids"), slice_len=2),
    batch_size=10,
)
rb

TensorDictReplayBuffer(
    storage=LazyTensorStorage(
        data=<empty>, 
        shape=None, 
        len=0, 
        max_size=100), 
    sampler=SliceSampler(num_slices=None, slice_len=2, end_key=('next', 'done'), traj_key=('collector', 'traj_ids'), truncated_key=('next', 'truncated'), strict_length=True), 
    writer=TensorDictRoundRobinWriter(cursor=0, full_storage=False), 
    batch_size=10, 
    collate_fn=<function _collate_id at 0x7f39e4b5fb00>)

In [136]:
data['collector','traj_ids']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [137]:
rb.extend(data)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [138]:
sample = rb.sample(10)

In [139]:
sample

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 2]), device=cpu, dtype=torch.int64, is_shared=False),
        action_value: Tensor(shape=torch.Size([10, 2]), device=cpu, dtype=torch.float32, is_shared=False),
        chosen_action_value: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        collector: TensorDict(
            fields={
                traj_ids: Tensor(shape=torch.Size([10]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        index: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10

In [140]:
sample['step_count']

tensor([[7],
        [8],
        [0],
        [1],
        [7],
        [8],
        [0],
        [1],
        [5],
        [6]])

In [141]:
sample["observation"]

tensor([[ 0.1190,  1.4084, -0.1088, -2.0614],
        [ 0.1472,  1.2145, -0.1500, -1.8043],
        [ 0.0313,  0.0413,  0.0107,  0.0229],
        [ 0.0322,  0.2362,  0.0111, -0.2663],
        [ 0.1190,  1.4084, -0.1088, -2.0614],
        [ 0.1472,  1.2145, -0.1500, -1.8043],
        [ 0.0313,  0.0413,  0.0107,  0.0229],
        [ 0.0322,  0.2362,  0.0111, -0.2663],
        [ 0.0745,  1.0168, -0.0451, -1.4403],
        [ 0.0948,  1.2125, -0.0739, -1.7467]])

In [142]:
sample["next", "observation"]

tensor([[ 0.1472,  1.2145, -0.1500, -1.8043],
        [ 0.1715,  1.4110, -0.1861, -2.1396],
        [ 0.0322,  0.2362,  0.0111, -0.2663],
        [ 0.0369,  0.4312,  0.0058, -0.5555],
        [ 0.1472,  1.2145, -0.1500, -1.8043],
        [ 0.1715,  1.4110, -0.1861, -2.1396],
        [ 0.0322,  0.2362,  0.0111, -0.2663],
        [ 0.0369,  0.4312,  0.0058, -0.5555],
        [ 0.0948,  1.2125, -0.0739, -1.7467],
        [ 0.1190,  1.4084, -0.1088, -2.0614]])

In [143]:
sample['representation']

tensor([[0.1495, 0.0832, 0.3370],
        [0.1411, 0.0853, 0.3108],
        [0.1717, 0.1063, 0.0000],
        [0.1425, 0.0952, 0.0061],
        [0.1495, 0.0832, 0.3370],
        [0.1411, 0.0853, 0.3108],
        [0.1717, 0.1063, 0.0000],
        [0.1425, 0.0952, 0.0061],
        [0.0857, 0.0904, 0.1998],
        [0.1152, 0.0904, 0.2713]])

In [144]:
sample

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 2]), device=cpu, dtype=torch.int64, is_shared=False),
        action_value: Tensor(shape=torch.Size([10, 2]), device=cpu, dtype=torch.float32, is_shared=False),
        chosen_action_value: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        collector: TensorDict(
            fields={
                traj_ids: Tensor(shape=torch.Size([10]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        index: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10

In [145]:
sample['representation']

first_states = sample[0::2] # even rows
second_states = sample[1::2] # odd rows (or next states)

print(first_states)
print(second_states)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([5, 2]), device=cpu, dtype=torch.int64, is_shared=False),
        action_value: Tensor(shape=torch.Size([5, 2]), device=cpu, dtype=torch.float32, is_shared=False),
        chosen_action_value: Tensor(shape=torch.Size([5, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        collector: TensorDict(
            fields={
                traj_ids: Tensor(shape=torch.Size([5]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([5]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([5, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        index: Tensor(shape=torch.Size([5, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([5, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([5, 4]), de

In [146]:
first_states['representation']

tensor([[0.1495, 0.0832, 0.3370],
        [0.1717, 0.1063, 0.0000],
        [0.1495, 0.0832, 0.3370],
        [0.1717, 0.1063, 0.0000],
        [0.0857, 0.0904, 0.1998]])

In [147]:
second_states['representation']

tensor([[0.1411, 0.0853, 0.3108],
        [0.1425, 0.0952, 0.0061],
        [0.1411, 0.0853, 0.3108],
        [0.1425, 0.0952, 0.0061],
        [0.1152, 0.0904, 0.2713]])

In [148]:
import torch

a = torch.tensor([1, 2, 3])
repeated_a = torch.Tensor.repeat(a, (2, 3))

print(repeated_a)

a = torch.tensor([1, 2, 3])
tiled_a = torch.tile(a, (2, 3))
print(tiled_a)

tensor([[1, 2, 3, 1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3, 1, 2, 3]])
tensor([[1, 2, 3, 1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3, 1, 2, 3]])


In [149]:
second_states['representation']

tensor([[0.1411, 0.0853, 0.3108],
        [0.1425, 0.0952, 0.0061],
        [0.1411, 0.0853, 0.3108],
        [0.1425, 0.0952, 0.0061],
        [0.1152, 0.0904, 0.2713]])

In [150]:
repeated_rep.shape

torch.Size([5, 5, 3])

In [151]:
second_states['representation'].shape # batch, rep_dim

repeated_rep = torch.tile(second_states['representation'], (1,1,5)).view(5,5,3)
repeated_rep

tensor([[[0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108]],

        [[0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061]],

        [[0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108]],

        [[0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061]],

        [[0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713]]])

In [152]:
second_states['representation'].shape

torch.Size([5, 3])

In [168]:
def squarify(x):
    # Squarify will take the input and adds a new dimension between the batch and the representation
    # so that the representation is repeated along the new dimension
    # To visualize thing of x as a matrix of batch_size x representation_dim
    # and squarify will place that matrix in a lateral way and repeat it along the new dimension j

    # NOTE: after squarify if you pick a i-th row all the elements (j-th index) in that row will be the same
    batch_size = x.shape[0]
    if len(x.shape) > 1:
        representation_dim = x.shape[-1]
        return x.tile((batch_size,)).view(batch_size, batch_size, representation_dim)
    return x.tile((batch_size,)).view(batch_size, batch_size)

In [170]:
squarify(second_states['next','reward']).squeeze(-1)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [161]:
squarify(second_states['representation'])

tensor([[[0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108]],

        [[0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061]],

        [[0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108],
         [0.1411, 0.0853, 0.3108]],

        [[0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061],
         [0.1425, 0.0952, 0.0061]],

        [[0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713],
         [0.1152, 0.0904, 0.2713]]])

In [113]:
def representation_distances(first_representations, second_representations,
                             distance_fn, beta=0.1,
                             return_distance_components=False):
  """Compute distances between representations.
     In the paper, it corresponds to the calculation of the U term
     for each pair of representations in the batch (all-vs-all).

  This will compute the distances between two representations.

  Args:
    first_representations: first set of representations to use.
    second_representations: second set of representations to use.
    distance_fn: function to use for computing representation distances.
    beta: float, weight given to cosine distance between representations.
    return_distance_components: bool, whether to return the components used for
      computing the distance.

  Returns:
    The distances between representations, combining the average of the norm of
    the representations and the distance given by distance_fn.
  """
  batch_size = first_representations.shape[0]
  representation_dim = first_representations.shape[-1]

  # Squarify the representations and reshape them to make a pair-waise comparison with vmap
  first_squared_reps = squarify(first_representations)
  first_squared_reps = torch.reshape(first_squared_reps,
                                   [batch_size**2, representation_dim])
  
  # Squarify the representations and reshape them to make a pair-waise comparison with vmap
  # However, we now need to permute (transpose) the dimension 0, 1 to alternate the values
  # so that we have the pair-wise comparisons of all-vs-all
  second_squared_reps = squarify(second_representations)
  second_squared_reps = torch.permute(second_squared_reps, dims=(1, 0, 2))
  second_squared_reps = torch.reshape(second_squared_reps,
                                    [batch_size**2, representation_dim])
  
  # vmap will calculate the pairwise distance_fn along the dimension specified
  # in in_axes. In this case, will take the dim 0 of the first_squared_reps and
  # the dim 0 of the second_squared_reps and apply the distance
  # It vertorize the process of calculating the distance between all the pairs

  # NOTE: base distance corresponds to the second term in the U calculation in the paper
  # It calculates the angle between the representations in the paper
  # Check what function is using
  base_distances = torch.vmap(distance_fn, in_dims=(0, 0))(first_squared_reps,
                                                         second_squared_reps)
  base_distances = base_distances
  print(base_distances.shape)
  # Sum along the second dimension and normalize the distance
  # NOTE: this is practically the first term of U in the paper
  norm_average = 0.5 * (torch.sum(torch.square(first_squared_reps), -1) +
                        torch.sum(torch.square(second_squared_reps), -1))
  
  print(norm_average.shape)
  if return_distance_components:
    return norm_average + beta * base_distances, norm_average, base_distances
  return norm_average + beta * base_distances

EPSILON = 1e-9

def _sqrt(x):
  # zeros like instead of zeros
  # It is because vmap works with a weird way of broadcasting
  # and a weird structure based on tensors
  tol = torch.zeros_like(x)
  return torch.sqrt(torch.maximum(x, tol))


def cosine_distance(x, y):
  # NOTE: the cosine similarity is not calculate directly for 
  # instabilities observed when using `jnp.arccos`, but I'm using torch
  # so I don't know if I will need to do this
  numerator = torch.sum(x * y)
  denominator = torch.sqrt(torch.sum(x**2)) * torch.sqrt(torch.sum(y**2))
  cos_similarity = numerator / (denominator + EPSILON)

  # cos_similarity = cos(theta)

  # NOTE: From, the Pythagorean trigometric identity
  # sin^2(theta) + cos^2(theta) = 1
  # you can get sin(theta) = sqrt(1 - cos^2(theta))
  # and the arctan2(sin(theta), cos(theta)) = theta
  return torch.arctan2(_sqrt(1. - cos_similarity**2), cos_similarity)

distances = representation_distances(first_states['representation'], 
                                     second_states['representation'], 
                                     cosine_distance)
distances

torch.Size([25])
torch.Size([25])


tensor([0.1357, 0.1928, 0.1357, 0.1928, 0.1266, 0.1906, 0.0400, 0.1906, 0.0400,
        0.1758, 0.1357, 0.1928, 0.1357, 0.1928, 0.1266, 0.1906, 0.0400, 0.1906,
        0.0400, 0.1758, 0.1046, 0.1417, 0.1046, 0.1417, 0.0849])

In [178]:
# NOTE: check in the main code if the output of this requires grad and the 
# other output must require grad
# @torch.no_grad()
def target_distances(representations, rewards, distance_fn, cumulative_gamma):
  """Target distance using the metric operator. This is the T in the paper :D"""
  next_state_similarities = representation_distances(
      representations, representations, distance_fn)
  squared_rews = squarify(rewards).squeeze(-1)
  squared_rews_transp = squared_rews.T
  squared_rews = squared_rews.reshape((squared_rews.shape[0]**2))
  squared_rews_transp = squared_rews_transp.reshape(
      (squared_rews_transp.shape[0]**2))
  reward_diffs = torch.abs(squared_rews - squared_rews_transp)
  return reward_diffs + cumulative_gamma * next_state_similarities

t_distances = target_distances(first_states['representation'], first_states['next','reward'], cosine_distance, cumulative_gamma = 0.9)

torch.Size([25])
torch.Size([25])


In [179]:
t_distances.requires_grad

False

In [26]:
# NOTE: Checar la distancia euclidean entre la representacion target y la representacion con
# la politica actual

# NOTE: en el repositorio de MICO, la distancia target es calculada con
# la target network (que es una copia de la politica actual) osea mis representaciones guardadas
# la distancia online por otra parte es calculada con una representacion con red actual
# y una representacion target


collector.policy(first_states['observation'])

(tensor([[0.1495, 0.0832, 0.3370],
         [0.1717, 0.1063, 0.0000],
         [0.1495, 0.0832, 0.3370],
         [0.1717, 0.1063, 0.0000],
         [0.0857, 0.0904, 0.1998]]),
 tensor([[0.0215, 0.2401],
         [0.0836, 0.2269],
         [0.0215, 0.2401],
         [0.0836, 0.2269],
         [0.0543, 0.2416]]),
 tensor([[0.2401],
         [0.2269],
         [0.2401],
         [0.2269],
         [0.2416]]),
 tensor([[0, 1],
         [0, 1],
         [0, 1],
         [0, 1],
         [0, 1]]))

In [27]:
import torch

# Create a sample tensor of shape [10, 3]
tensor = torch.randn(10, 3)

# Separate odd and even rows
even_rows = tensor[0::2]
odd_rows = tensor[1::2]

print("Even rows:\n", even_rows)
print("Odd rows:\n", odd_rows)


Even rows:
 tensor([[ 0.0062, -1.4055,  0.1195],
        [-0.3586,  1.7216, -0.2407],
        [-0.1195,  0.3416,  1.0750],
        [ 0.0381, -1.3240,  0.5548],
        [ 0.4521,  0.8146, -0.1394]])
Odd rows:
 tensor([[-0.3312, -0.8407,  0.2731],
        [-0.6251,  0.8161, -0.5711],
        [ 0.0508, -1.6557, -0.4615],
        [-1.0817,  2.1221,  0.0500],
        [-0.3677, -0.4574, -0.7223]])


In [28]:
loss = DQNLoss(value_network=policy, 
               action_space=env.action_spec, 
               delay_value=True) # delay_value=True means we will use a target network
optim = Adam(loss.parameters(), lr=0.02)

# eps: will be used to update the target network as 
# \theta_t = \theta_{t-1} * \epsilon + \theta_t * (1-\epsilon)
# where eps = 1 is hard update
updater = SoftUpdate(loss, eps=0.99)

In [34]:
loss.value_network_params

TensorDictParams(params=TensorDict(
    fields={
        module: TensorDict(
            fields={
                0: TensorDict(
                    fields={
                        module: TensorDict(
                            fields={
                                activation: TensorDict(
                                    fields={
                                    },
                                    batch_size=torch.Size([]),
                                    device=None,
                                    is_shared=False),
                                encoder: TensorDict(
                                    fields={
                                        0: TensorDict(
                                            fields={
                                                bias: Parameter(shape=torch.Size([64]), device=cpu, dtype=torch.float32, is_shared=False),
                                                weight: Parameter(shape=torch.Size([64, 4]), device=cpu, dtype

In [33]:
loss.target_value_network_params

TensorDictParams(params=TensorDict(
    fields={
        module: TensorDict(
            fields={
                0: TensorDict(
                    fields={
                        module: TensorDict(
                            fields={
                                activation: TensorDict(
                                    fields={
                                    },
                                    batch_size=torch.Size([]),
                                    device=None,
                                    is_shared=False),
                                encoder: TensorDict(
                                    fields={
                                        0: TensorDict(
                                            fields={
                                                bias: Parameter(shape=torch.Size([64]), device=cpu, dtype=torch.float32, is_shared=False),
                                                weight: Parameter(shape=torch.Size([64, 4]), device=cpu, dtype

In [80]:
from tensordict import TensorDict
from torchrl.data import SliceSampler
from torchrl.data import LazyMemmapStorage

rb = TensorDictReplayBuffer(
    storage=LazyMemmapStorage(size),
    sampler=SliceSampler(traj_key="episode", num_slices=4),
    batch_size=8,
)
episode = torch.zeros(10, dtype=torch.int)
episode[:3] = 1
episode[3:5] = 2
episode[5:7] = 3
episode[7:] = 4
steps = torch.cat([torch.arange(3), torch.arange(2), torch.arange(2), torch.arange(3)])
obs = torch.randn((3, 4, 5)).expand(10, 3, 4, 5)
data = TensorDict(
    {
        "episode": episode,
        "obs": obs,
        "act": torch.randn((20,)).expand(10, 20),
        "other": torch.randn((20, 50)).expand(10, 20, 50),
        "steps": steps,
    },
    [10],
)
rb.extend(data)
sample = rb.sample()
print("episode are grouped", sample["episode"])
print("steps are successive", sample["steps"])

episode are grouped tensor([3, 3, 4, 4, 2, 2, 1, 1], dtype=torch.int32)
steps are successive tensor([0, 1, 0, 1, 0, 1, 0, 1])


In [82]:
episode

tensor([1, 1, 1, 2, 2, 3, 3, 4, 4, 4], dtype=torch.int32)

In [81]:
rb

TensorDictReplayBuffer(
    storage=LazyMemmapStorage(
        data=TensorDict(
            fields={
                act: MemoryMappedTensor(shape=torch.Size([10, 20]), device=cpu, dtype=torch.float32, is_shared=False),
                episode: MemoryMappedTensor(shape=torch.Size([10]), device=cpu, dtype=torch.int32, is_shared=False),
                index: MemoryMappedTensor(shape=torch.Size([10]), device=cpu, dtype=torch.int64, is_shared=False),
                obs: MemoryMappedTensor(shape=torch.Size([10, 3, 4, 5]), device=cpu, dtype=torch.float32, is_shared=False),
                other: MemoryMappedTensor(shape=torch.Size([10, 20, 50]), device=cpu, dtype=torch.float32, is_shared=False),
                steps: MemoryMappedTensor(shape=torch.Size([10]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared=False), 
        shape=torch.Size([10]), 
        len=10, 
        max_size=100), 
    sampler=S

In [234]:
import datetime

current_date = datetime.datetime.now()
date_str = current_date.strftime("%Y_%m_%d-%H_%M_%S")  # Includes date and time
date_str

'2024_07_23-17_34_50'

In [235]:
total_count = 0
total_episodes = 0
t0 = time.time()
for i, data in enumerate(collector):
    # Write data in replay buffer
    rb.extend(data)
    max_length = rb[:]["next", "step_count"].max() # From all the next steps get the max step count
    if len(rb) > init_rand_steps: # wam-up steps
        # Optim loop (we do several optim steps
        # per batch collected for efficiency)
        for _ in range(optim_steps):
            sample = rb.sample(128) # sample a batch of 128 (repetition is allowed)
            # print(sample)
            break
            loss_vals = loss(sample)
            loss_vals["loss"].backward()
            optim.step()
            optim.zero_grad()
            # Update exploration factor
            # NOTE: Why I am updating the exploration factor here? 
            # I'm considering practically that I did 100 (or n) iteractions in the environment time optim_steps
            exploration_module.step(data.numel()) # data.numel() returns the number of elements in the data
            # Update target params each optimisation step
            updater.step()
            if i % 10:
                torchrl_logger.info(f"Max num steps: {max_length}, rb length {len(rb)}")
            total_count += data.numel()
            total_episodes += data["next", "done"].sum() # sum the number of done episodes
    
    if max_length > 200:
        break

t1 = time.time()

torchrl_logger.info(
    f"solved after {total_count} steps, {total_episodes} episodes and in {t1-t0}s."
)

/home/zosov/anaconda3/envs/final-project/lib/python3.11/site-packages/torchrl/data/replay_buffers/replay_buffers.py:585: UserWarning: Got conflicting batch_sizes in constructor (32) and `sample` (128). Refer to the ReplayBuffer documentation for a proper usage of the batch-size arguments. The batch-size provided to the sample method will prevail.
  warnings.warn(
2024-07-23 17:34:53,433 [torchrl][INFO] solved after 0 steps, 0 episodes and in 2.57519268989563s.


In [242]:
sample[0]

TensorDict(
    fields={
        _weight: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.float32, is_shared=False),
        action: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.int64, is_shared=False),
        action_value: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        chosen_action_value: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        collector: TensorDict(
            fields={
                traj_ids: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        index: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.int64, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_sha

In [236]:
sample

TensorDict(
    fields={
        _weight: Tensor(shape=torch.Size([128]), device=cpu, dtype=torch.float32, is_shared=False),
        action: Tensor(shape=torch.Size([128, 2]), device=cpu, dtype=torch.int64, is_shared=False),
        action_value: Tensor(shape=torch.Size([128, 2]), device=cpu, dtype=torch.float32, is_shared=False),
        chosen_action_value: Tensor(shape=torch.Size([128, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        collector: TensorDict(
            fields={
                traj_ids: Tensor(shape=torch.Size([128]), device=cpu, dtype=torch.int64, is_shared=False)},
            batch_size=torch.Size([128]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([128, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        index: Tensor(shape=torch.Size([128]), device=cpu, dtype=torch.int64, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([128, 1]),

In [8]:
record_env.rollout(max_steps=1000, policy=policy)
video_recorder.dump()

In [5]:
import random

# Generate and print 10 random seeds
random_seeds = [random.randint(0, 1000000) for _ in range(10)]
print(random_seeds)

[118398, 676190, 786456, 171936, 887739, 919409, 711872, 442081, 189061, 117840]
